In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [37]:
df = sns.load_dataset('tips')
df


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [38]:
#predict what is the time? lunch dinner ? time is traget
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [39]:
#Eda automation
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [40]:
#time is a nominal variable, label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['time']= le.fit_transform(df.time)

In [41]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [42]:
df.time.unique()
#dinner 1 luncjh 0

array([0, 1])

In [43]:
X = df.drop(['time'], axis=1)
y = df['time']
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [44]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((195, 6), (49, 6), (195,), (49,))

In [47]:
X_train.head()

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2


In [48]:
#handling missing values
#data encoding
#feature scaling

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline #a sequence of data transformer
from sklearn.compose import ColumnTransformer #groups all the pipeline steps in each column 


In [49]:
cat_cols = ['sex', 'smoker', 'day']
num_cols = ['total_bill', 'size', 'tip']
#num cols
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler()),  # Feature scaling
    
])

#cat cols
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('encoder', OneHotEncoder(drop='first'))  # One-hot encoding
])

In [50]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),  # Apply num_pipeline to num_cols
        ('cat', cat_pipeline, cat_cols)   # Apply cat_pipeline to cat_cols
    ]
)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'size', 'tip']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['sex', 'smoker', 'day'])])

In [52]:
X_train = preprocessor.fit_transform(X_train)

In [53]:
X_test = preprocessor.transform(X_test)

In [55]:
#data is ready for model training
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [57]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVC' : SVC(),
}
models

{'Logistic Regression': LogisticRegression(),
 'Decision Tree': DecisionTreeClassifier(),
 'SVC': SVC()}

In [60]:
for i in range(len(models)):
    print(list(models.values())[i])


LogisticRegression()
DecisionTreeClassifier()
SVC()


In [61]:
from sklearn.metrics import accuracy_score
def model_train_eval(X_train, X_test, y_train, y_test, models):
    evaluation= {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        evaluation[list(models.keys())[i]] = acc
    return evaluation


In [62]:
model_train_eval(X_train, X_test, y_train, y_test, models)

{'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9795918367346939}

In [63]:
from sklearn.ensemble import RandomForestClassifier
rf  = RandomForestClassifier()
rf

RandomForestClassifier()

In [68]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
params = {
    'max_depth': [1, 2, 3, 4, 5, 6, 10, None],
    'n_estimators': [30, 40, 50,100,200,300],
    'criterion': ['gini', 'entropy']
}
params

{'max_depth': [1, 2, 3, 4, 5, 6, 10, None],
 'n_estimators': [30, 40, 50, 100, 200, 300],
 'criterion': ['gini', 'entropy']}

In [69]:
clf = RandomizedSearchCV(rf, param_distributions=params, cv = 5, verbose = 3, scoring = 'accuracy', n_iter=10)

In [70]:
clf

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 10,
                                                      None],
                                        'n_estimators': [30, 40, 50, 100, 200,
                                                         300]},
                   scoring='accuracy', verbose=3)

In [71]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=40;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=40;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=40;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=40;, score=0.923 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=40;, score=0.923 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.974 total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.923 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.949 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 10,
                                                      None],
                                        'n_estimators': [30, 40, 50, 100, 200,
                                                         300]},
                   scoring='accuracy', verbose=3)

In [72]:
clf.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'entropy'}

In [73]:
clf.best_score_

np.float64(0.9589743589743589)

In [75]:
final_model = clf.best_estimator_
y_pred = final_model.predict(X_test)
y_pred

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1])

In [76]:
accuracy_score(y_test, y_pred)

1.0